Автор: Алексеева Валентина

In [ ]:
! pip install datasets docx2txt langchain langchain_community networkx

In [41]:
import openai
import numpy as np
from typing import List

import torch
import torch.nn as nn
import torch.nn.functional as F

from pathlib import Path
import fitz
from docx import Document
import re

import networkx as nx
from tqdm.auto import tqdm
import pandas as pd

from bert_score import score

from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

import matplotlib.pyplot as plt
import torch.optim as optim

## Инициализация клиента и моделей

In [28]:
client = openai.OpenAI(
    api_key="1111111",
    base_url="https://api.vsellm.ru/v1"
)

embed_model_name = 'openai/text-embedding-3-small'
generative_model_name = "openai/gpt-4.1-mini"

## Пример использования генеративных моделей

Генерация эмбедингов

In [31]:
texts = [
    "Graph Neural Networks are powerful",
    "GNNs are used in recommender systems",
    "Transformers work well for NLP"
]

response = client.embeddings.create(
    model=embed_model_name,
    input=texts
)

embeddings = [item.embedding for item in response.data]

print(len(embeddings))          # количество текстов
print(len(embeddings[0]))       # размерность эмбеддинга (3072)


3
1536


Генерация текста

In [32]:
response = client.chat.completions.create(
    model=generative_model_name,
    messages=[
        {"role": "user", "content": "Привет!"}
    ]
)

print(response.choices[0].message.content)


Привет! Как я могу помочь?


## Baseline HACK2025


## Загрузка базы знаний

In [29]:
def load_docx(file_path):
    doc = Document(file_path)
    text = "\n".join([p.text for p in doc.paragraphs])
    return re.sub(r'\s+', ' ', text).strip()

def load_pdf(file_path):
    text = ""
    with fitz.open(file_path) as pdf:
        for page in pdf:
            text += page.get_text("text") + "\n"
    return re.sub(r'\s+', ' ', text).strip()

data_folder = Path("lectures") 

texts = []
for file in data_folder.glob("*"):
    if file.suffix == ".docx":
        print(f"Загрузка DOCX: {file.name}")
        texts.append(load_docx(file))
    elif file.suffix == ".pdf":
        print(f"Загрузка PDF: {file.name}")
        texts.append(load_pdf(file))
        
print(f"\nНайдено файлов: {len(texts)}")

full_text = "\n".join(texts)
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_text(full_text)

print(f"Total chunks: {len(chunks)}")

Загрузка PDF: GNN_module1.pdf
Загрузка PDF: GNN_module2.pdf
Загрузка PDF: module3.pdf
Загрузка PDF: module4.pdf
Загрузка PDF: module5.pdf
Загрузка PDF: module6.pdf
Загрузка PDF: module7.pdf
Загрузка PDF: module8.pdf
Загрузка DOCX: Курс.docx

Найдено файлов: 9
Total chunks: 380


## Генерация эмбеддингов для чанков

In [30]:
# обновлено для работы не только с docx
def embed_documents(texts):
    embeddings = []
    for i in tqdm(range(0, len(texts), 50)):  # батчами по 50
        batch = texts[i:i+50]
        response = client.embeddings.create(
            model=embed_model_name,
            input=batch
        )
        embeddings.extend([item.embedding for item in response.data])
    return np.array(embeddings)

chunk_embeddings = embed_documents(chunks)
print("Эмбеддинги сгенерированы:", chunk_embeddings.shape)
print("Пример:", chunk_embeddings[0][:5])

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:07<00:00,  1.11it/s]

Эмбеддинги сгенерированы: (380, 1536)
Пример: [-0.03986635 -0.04760197  0.00567065 -0.00929089  0.04135756]


## Retrieval (cosine similarity)

In [31]:
def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


def retrieve(query, k=5):
    query_emb = client.embeddings.create(
        model=embed_model_name,
        input=[query]
    ).data[0].embedding

    scores = [
        cosine_sim(query_emb, emb)
        for emb in chunk_embeddings
    ]

    top_idx = np.argsort(scores)[-k:][::-1]

    return [
        {
            "text": chunks[i].page_content,
            "metadata": chunks[i].metadata,
            "score": float(scores[i])
        }
        for i in top_idx
    ]


# Построение графа

In [32]:
def build_graph(embeddings, threshold=0.75, k=5):
    """
    embeddings : np.ndarray, размер (n_chunks, emb_dim)
    threshold  : минимальная косинусная близость для ребра
    k          : сколько соседей связывать
    """
    n = len(embeddings)
    G = nx.Graph()
    G.add_nodes_from(range(n))

    for i in tqdm(range(n)):
        sims = np.dot(embeddings, embeddings[i]) / (
            np.linalg.norm(embeddings, axis=1) * np.linalg.norm(embeddings[i]) + 1e-10
        )
        top_idx = np.argsort(sims)[-k-1:-1][::-1]
        for j in top_idx:
            if sims[j] > threshold:
                G.add_edge(i, j, weight=float(sims[j]))
    print(f"Граф построен: {len(G.nodes())} узлов, {len(G.edges())} рёбер")
    return G

G = build_graph(chunk_embeddings, threshold=0.75, k=5)

100%|███████████████████████████████████████████████████████████████████████████████| 380/380 [00:01<00:00, 247.98it/s]

Граф построен: 380 узлов, 141 рёбер


# Матрица смежности

In [33]:
A = nx.to_numpy_array(G)
A = torch.tensor(A, dtype=torch.float32)
I = torch.eye(A.shape[0])
A_hat = A + I
D_hat = torch.diag(torch.sum(A_hat, dim=1) ** -0.5)
A_norm = D_hat @ A_hat @ D_hat   # симметрическая нормализация

# Модель GraphSAGE
GraphSAGE обновит вектор каждого узла, учитывая соседние чанки. По сути, он "распространяет" знания по графу. Это делает retrieval гораздо умнее: он теперь знает, какие чанки логически связаны.

* `A_norm` — нормализованная матрица смежности (учитывает связи между чанками).
* `GraphSAGE` — два линейных слоя, распространяющих признаки по графу (аналог DGL версии).
* Выдаёт новые векторы размером `(380, 128)`: они уже "обогащены контекстом соседей"

In [34]:
class GraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.W1 = nn.Linear(in_dim, hidden_dim)
        self.W2 = nn.Linear(hidden_dim, out_dim)

    def forward(self, X, A_norm):
        H = A_norm @ X
        H = F.relu(self.W1(H))
        H = A_norm @ H
        H = self.W2(H)
        return H
        
X = torch.tensor(chunk_embeddings, dtype=torch.float32)
model = GraphSAGE(X.shape[1], 256, 128)

with torch.no_grad():
    gnn_embeddings = model(X, A_norm).numpy()

print("Новые эмбеддинги:", gnn_embeddings.shape)

Новые эмбеддинги: (380, 128)


## Сбор контекста

План:
* Получать embedding запроса;
* Искать ближайшие чанки (по cosine similarity);
* Добавлять соседей из графа (multi-hop);
* Передавать собранный контекст в LLM для генерации ответа.

In [35]:
def project_query_embedding(query_emb):
    q = torch.tensor(query_emb, dtype=torch.float32)
    q_proj = torch.matmul(q, W_proj)
    q_proj = q_proj / torch.norm(q_proj)  # нормализация
    return q_proj.numpy()
    
# Ретривер по GNN-эмбеддингам
def retrieve_gnn(query, k=5):
    query_emb = client.embeddings.create(
        model=embed_model_name,
        input=[query]
    ).data[0].embedding

    # проекция в 128-мерное пространство
    query_proj = project_query_embedding(query_emb)

    scores = np.dot(gnn_embeddings, query_proj) / (
        np.linalg.norm(gnn_embeddings, axis=1) * np.linalg.norm(query_proj) + 1e-10
    )

    top_idx = np.argsort(scores)[-k:][::-1]
    return top_idx, scores

# Преобразование query-эмбеддинга к размерности 128
W_proj = torch.randn((1536, 128), dtype=torch.float32) / np.sqrt(1536)    

In [36]:
# Multi-hop расширение по графу
def expand_multi_hop(G, node_indices, hops=2):
    expanded = set(node_indices)
    for _ in range(hops):
        for node in list(expanded):
            expanded.update(G.neighbors(node))
    return list(expanded)

# Сбор контекста
def build_context(G, query, k=5, hops=2):
    top_idx, scores = retrieve_gnn(query, k)
    expanded_idx = expand_multi_hop(G, top_idx, hops)
    expanded_idx = list(set(expanded_idx))
    context = "\n\n".join(chunks[i] for i in expanded_idx)
    return context

In [46]:
# пример
query = "Что такое Graph Neural Network?"
context = build_context(G, query, k=5, hops=2)
print(context[:500])

конкатенации, - 𝐿𝑒𝑎𝑘𝑦𝑅𝑒𝐿𝑈— нелинейная функция активации. 𝑒"! измеряет, насколько "похожи" или "релевантны" ℎ" и ℎ!, и используется для определения значимости узла 𝑢 для узла 𝑣. Graph Neural Networks(GAT) Нормализация внимания. Для получения финального веса внимания 𝛼"!, нормализуем 𝑒"! через softmax: 𝛼"! = 𝑒𝑥𝑝𝑒"! ∑6∈2 " 𝑒𝑥𝑝𝑒"6 где: - 𝑁𝑣— множество соседей узла 𝑣 - 𝛼"! показывает относительный вклад узла 𝑢в обновление узла 𝑣. 39 Graph Neural Networks(GAT) Агрегация соседей Признаки узла

из огром


## Генерация ответа

In [37]:
def generate_answer(question, context):
    response = client.chat.completions.create(
        model=generative_model_name,
        messages=[
            {
                "role": "system",
                "content": (
                    "Ты ассистент, отвечающий на вопросы по курсу Graph Neural Networks. "
                    "Используй только предоставленный контекст."
                )
            },
            {
                "role": "user",
                "content": f"""
Контекст:
{context}

Вопрос:
{question}

Дай краткий и точный ответ.
"""
            }
        ],
        temperature=0.2
    )

    return response.choices[0].message.content


## Финальная функция answer()

In [38]:
def answer(question, k=5, hops=2):
    context = build_context(G, question, k=k, hops=hops)
    answer_text = generate_answer(question, context)

    return {
        "question": question,
        "answer": answer_text,
        "context_length": len(context.split()),
    }

In [39]:
result = answer("Что такое Graph Neural Network?")

print("Вопрос:", result["question"])
print("Ответ:\n", result["answer"])
print(f"Длина контекста: {result['context_length']} слов")

Вопрос: Что такое Graph Neural Network?
Ответ:
 Graph Neural Network (GNN) — это модель, которая обрабатывает графовые данные, учитывая структуру графа и свойства узлов и ребер, с помощью механизмов, таких как внимание (attention), для динамического взвешивания соседей и обновления эмбеддингов узлов. GNN применяется для задач, где данные представлены в виде графов, например, в рекомендационных системах, обнаружении мошенничества, молекулярном моделировании и анализе социальных сетей.
Длина контекста: 593 слов


In [48]:
qa_df = pd.read_excel("QA_dataset.xlsx")
questions = qa_df["Question"].dropna().tolist()
answers = []

for q in tqdm(questions):
    res = answer(q, k=5, hops=2)
    answers.append(res["answer"])

qa_df["Answer"] = answers
qa_df.to_excel("QA_dataset_answered.xlsx", index=False)

qa_df.head()

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [08:57<00:00,  5.38s/it]


,Question,Answer
0,Что такое граф и как он формально задаётся?,"Граф — это структура данных, формально задавае..."
1,Какие основные компоненты используются для опи...,Основные компоненты для описания динамического...
2,В чём отличие snapshot-based и event-based дин...,Snapshot-based динамика представляет граф как ...
3,Какие задачи решают динамические GNN?,Динамические GNN решают задачи прогнозирования...
4,Почему для динамических графов требуется памят...,Для динамических графов требуется память узлов...


# Сравнение с baseline

Подсчет `bert-score` выполнен с использованием ресурсов KAGGLE, код в репрезитории, результат сохранен в `QA_dataset_comparison.xlsx`

BERT_F1 ≈ Семантическое сходство между Graph-RAG и Baseline ответом

| Диапазон F1 | Интерпретация                                                 |
| ----------- | ------------------------------------------------------------- |
| 0.9–1.0     | Ответы почти идентичны                                        |
| 0.8–0.9     | Ответы очень похожи, формулировка изменилась незначительно    |
| 0.7–0.8     | Смысл похож, но Graph-RAG переформулировал / добавил контекст |
| 0.6–0.7     | Graph-RAG дал существенно другой ответ                    |
| < 0.6       | Ответы разные по смыслу                                   |


In [14]:
compare_df = pd.read_excel("QA_dataset_comparison.xlsx")

mean_f1 = compare_df["BERT_F1"].mean()
std_f1 = compare_df["BERT_F1"].std()
max_f1 = compare_df["BERT_F1"].max()
min_f1 = compare_df["BERT_F1"].min()

print("Метрики по BERT_F1:")
print(f"Среднее значение: {mean_f1:.4f}")
print(f"Стандартное отклонение: {std_f1:.4f}")
print(f"Максимум: {max_f1:.4f}")
print(f"Минимум: {min_f1:.4f}")

compare_df["Improvement"] = compare_df["BERT_F1"] - mean_f1

Метрики по BERT_F1:
Среднее значение: 0.7846
Стандартное отклонение: 0.0906
Максимум: 1.0000
Минимум: 0.6144


Все ответы с использованием GNN дали ответы по BERT_F1 (в сравненнии с baselinе без GNN) выше 0.60, а в среднем 0.75-0.80. Это означает, что ответы похожи смыслу с baseline, но не индентичны ему. Логика ответов не ломается, лишь появляются переформулировки и дополнительный контекст

In [24]:
noinfo_pattern = re.compile(r"предоставленном контексте", flags=re.IGNORECASE)

compare_df["Baseline_noinfo"] = compare_df["Baseline_Answer"].apply(lambda x: bool(noinfo_pattern.search(str(x))))
compare_df["Graph_noinfo"] = compare_df["Graph_Answer"].apply(lambda x: bool(noinfo_pattern.search(str(x))))

total = len(compare_df)
baseline_noinfo = compare_df["Baseline_noinfo"].sum()
graph_noinfo = compare_df["Graph_noinfo"].sum()
both_noinfo = ((compare_df["Baseline_noinfo"]) & (compare_df["Graph_noinfo"])).sum()

print(f"Всего вопросов: {total}")
print(f"Baseline не ответил: {baseline_noinfo} ({baseline_noinfo/total:.1%})")
print(f"Graph-RAG не ответил: {graph_noinfo} ({graph_noinfo/total:.1%})")
print(f"Оба не ответили: {both_noinfo} ({both_noinfo/total:.1%})")

# Примеры 

print("Примеры, где оба не ответили:")
both_examples = compare_df[compare_df["Baseline_noinfo"] & compare_df["Graph_noinfo"]].head(5)
for i, row in both_examples.iterrows():
    print(f"{row['Question']}")
    print(f"Baseline: {row['Baseline_Answer'][:200]}...")
    print(f"Graph-RAG: {row['Graph_Answer'][:200]}...")
    print(f"BERT_F1 = {row['BERT_F1']:.4f}")

print("\nПримеры, где Graph-RAG ответил, а Baseline нет:")
graph_only = compare_df[(~compare_df["Graph_noinfo"]) & (compare_df["Baseline_noinfo"])].head(5)
for i, row in graph_only.iterrows():
    print(f"{row['Question']}")
    print(f"Baseline: {row['Baseline_Answer'][:200]}...")
    print(f"Graph-RAG: {row['Graph_Answer'][:200]}...")
    print(f"BERT_F1 = {row['BERT_F1']:.4f}")

print("\nПримеры, где Baseline ответил, а Graph-RAG нет:")
base_only = compare_df[(compare_df["Graph_noinfo"]) & (~compare_df["Baseline_noinfo"])].head(5)
for i, row in base_only.iterrows():
    print(f"{row['Question']}")
    print(f"Baseline: {row['Baseline_Answer'][:200]}...")
    print(f"Graph-RAG: {row['Graph_Answer'][:200]}...")
    print(f"BERT_F1 = {row['BERT_F1']:.4f}")

Всего вопросов: 100
Baseline не ответил: 16 (16.0%)
Graph-RAG не ответил: 31 (31.0%)
Оба не ответили: 14 (14.0%)
Примеры, где оба не ответили:
Какие результаты показал PinSAGE на датасете Amazon?
Baseline: В предоставленном контексте нет информации о результатах PinSAGE на датасете Amazon....
Graph-RAG: В предоставленном контексте нет информации о результатах PinSAGE на датасете Amazon....
BERT_F1 = 1.0000
Какие prompt-engineering техники лучше всего работают для Graph-RAG?
Baseline: В предоставленном контексте нет информации о prompt-engineering техниках для Graph-RAG....
Graph-RAG: В предоставленном контексте нет информации о prompt-engineering техниках для Graph-RAG....
BERT_F1 = 1.0000
Какие значения learning rate использовались в оригинальной статье GraphMAE?
Baseline: В предоставленном контексте нет информации о значениях learning rate, использованных в оригинальной статье GraphMAE....
Graph-RAG: В предоставленном контексте нет информации о значениях learning rate, использованных

Данные метрики не дают прямой информации о качестве ответов, лишь об отличиях от baseline. Не исключено, что среди ответов обеих моделей есть неправильные/неточные или что информации о теме на самом деле нет в источниках, а модели все равно "подсмотрели" куда-либо еще.

## Описание решения: Graph-RAG + GNN

### Построение графа

После разбиения учебных материалов на смысловые чанки (около 380 фрагментов текста) был построен взвешенный граф взаимосвязей между ними. Каждый узел соответствует текстовому чанку, а ребро добавляется между узлами, если косинусное сходство их эмбеддингов (из модели `text-embedding-3-small`) превышает заданный порог (например, 0.8).

Таким образом формируется граф `G = (V, E)`, где:

* `|V| = 380` — количество чанков (вершин);
* `|E| ≈ 140` — количество семантических связей;
* веса рёбер отражают силу смыслового родства между фрагментами.

Анализ структуры графа позволяет находить семантически близкие, но не явно соседние части текста, а также выполнять multi-hop retrieval, учитывая контекст на 2–3 перехода вперёд.

### Используемые модели GNN

#### GNN базовая GraphSAGE

Модель использует двухслойную архитектуру GraphSAGE с mean-агрегацией:

$$h_v^{(k+1)} = \sigma(W^{(k)} \cdot \text{AGGREGATE}({h_v^{(k)}} \cup {h_u^{(k)} | u \in N(v)}))$$

* Вход: эмбеддинги чанков из `text-embedding-3-small`
* Цель: обучить представления так, чтобы соседние чанки имели близкие векторы
* Функция потерь: контрастивная функция, минимизирующая различие между соседями

Результат новые графовые эмбеддинги размерности 128, учитывающие структуру документа и смысловые связи.


### Как граф улучшает Retrieval-Augmented Generation

Граф используется на этапе retrieval для усиления поиска релевантных чанков для LLM.

1. Baseline RAG: выбирает top-k чанков по косинусному сходству без контекста.
2. Graph-RAG (с GNN):

   * преобразует все чанки в графовые эмбеддинги;
   * извлекает top-k кандидатов по GNN-пространству;
   * выполняет multi-hop расширение (2 шага по графу);
   * формирует расширенный контекст для генерации ответа.

Это позволяет LLM использовать логически связанные фрагменты из разных частей курса, а не только локальные совпадения. 


#### Где можно вдохновиться: RAG From Scratch